## 🎯 Prompt Engineering Fundamentals

Welcome to the first session of our AI Workshop! In this notebook, we'll dive deep into **Prompt Engineering** - the art and science of crafting effective instructions for Large Language Models (LLMs).

### 🎯 Learning Objectives
By the end of this notebook, you will understand:
- The fundamentals of prompt engineering
- Different prompting strategies: Zero-Shot, One-Shot, and Few-Shot
- How to format responses using examples
- Best practices for structuring prompts with conversation roles
- Practical applications in classification and conversational AI

### 📚 What is Prompt Engineering?
Prompt engineering is the process of designing and optimizing text prompts to get the best possible responses from AI language models. It's like learning how to ask the right questions in the right way to get accurate, useful, and well-formatted answers.

Think of it as the bridge between human intent and AI understanding - the better your prompts, the better your results!

---

### 📦 Installing Required Packages
We'll install the OpenAI library for Azure OpenAI integration and Panel for interactive widgets.

In [ ]:
%pip install openai

In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key = os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version = os.getenv("AZURE_OPENAI_API_VERSION")
)

print(os.getenv("AZURE_OPENAI_ENDPOINT"))

### 🔑 Setting up Azure OpenAI Client
Here we initialize our connection to Azure OpenAI. Make sure you have set the following environment variables:
- `AZURE_OPENAI_ENDPOINT`: Your Azure OpenAI service endpoint
- `AZURE_OPENAI_KEY`: Your Azure OpenAI API key

💡 **Pro Tip**: Never hardcode API keys in your code! Always use environment variables for security.

# 🎭 Few-Shot Learning: Teaching AI by Example

## The Power of Examples in Prompt Engineering

To obtain the model's response in a specific format, we have various options, but one of the most powerful is to use **Few-Shot Learning**. This involves presenting the model with pairs of user queries and example responses.

### 📊 Understanding Shot Types

Large models like GPT respond remarkably well to the examples provided, adapting their responses to match the specified format and style. The terminology depends on the number of examples:

- **🎯 Zero-Shot**: No examples provided - the model relies purely on its training and your instructions
- **🎯 One-Shot**: A single example to demonstrate the desired format
- **🎯 Few-Shot**: Multiple examples (typically 2-6) to establish a clear pattern

### 🎓 Best Practices
- **One shot is often enough** for most tasks
- **Maximum of 6 shots** is recommended for optimal performance
- **Remember**: Examples consume input tokens on every request
- **Quality over quantity**: Better examples lead to better results

Let's explore each approach with practical examples!

In [ ]:
#Function to call the model.
def return_OAIResponse(user_message, context):

#As we can see, we’re adding the user’s question at the end of the prompt with the user role, so the model understands that this is a user request and not an instruction on how it should work.
    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    # print(newcontext)

    response = client.chat.completions.create(
        model="gpt-4.1", # model = "deployment_name".
        messages=newcontext,
        temperature=0,
        max_tokens=800
    )

    #print(response)
    # print(response.model_dump_json(indent=2))
    # print(response.choices[0].message.content)

    return (response.choices[0].message.content)
    #return (response.choices[0].message["content"])

### 🔧 Helper Function: Our AI Communication Bridge

Before we start experimenting with different prompting techniques, let's create a reusable function that will handle our conversations with the AI model.

This function will:
1. Take your message and context (previous conversation)
2. Format it properly for the AI model
3. Send the request to Azure OpenAI
4. Return the AI's response

**Key Features:**
- Automatically adds user role to messages
- Uses consistent temperature settings for reproducible results
- Handles the Azure OpenAI API communication

## 🎯 Zero-Shot Prompting: The Minimalist Approach

**Zero-Shot prompting** is the simplest form of prompt engineering. We provide no examples - just a clear instruction about what the AI should do.

### 🔍 What to Expect:
- ✅ **Correct information** based on the model's training data
- ❌ **No specific formatting** - the model uses its default response style
- 🎲 **Variable output format** - responses may vary in structure

This approach works well when you need quick, accurate information but don't care about the specific format of the response.

In [ ]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

### 📝 Zero-Shot Example: F1 Championship Query
Let's ask about F1 champions with minimal context - just telling the AI it's an F1 expert.

In [ ]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

## 🎯 One-Shot Prompting: Learning from a Single Example

**One-Shot prompting** provides exactly one example to show the AI the desired format. This is often the sweet spot between simplicity and control.

### 🔍 What to Expect:
- ✅ **Structured response** following your example format
- ✅ **Consistent formatting** across multiple queries
- ✅ **Efficient token usage** - only one example needed

Notice how we embed the example directly in the system message. This teaches the AI the pattern we want to follow.

## 🎯 Few-Shot Prompting: Multiple Examples for Complex Patterns

For more complex formats or when working with smaller/less capable models, **Few-Shot prompting** with multiple examples can be very effective.

### 🔍 When to Use Few-Shot:
- 📋 **Complex formatting requirements**
- 🤖 **Smaller or specialized models**
- 🎯 **High consistency needs**
- 📊 **Structured data extraction**

Let's see how multiple examples reinforce the pattern and improve consistency.

In [ ]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Bettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

In [ ]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

### 🔄 Testing Pattern Consistency
Let's test if our few-shot examples create a consistent pattern by asking about a different year.

## 🎭 Advanced Few-Shot: Using Conversation Roles

### 🚀 The Recommended Approach

While embedding examples in system messages works, the **proper way** is to use OpenAI's conversation roles. This approach is more effective because:

- 🎓 **Natural Learning**: The model learns from realistic conversation patterns
- 🔄 **Better Context Understanding**: Roles help the AI understand the conversation flow
- ⚡ **Improved Performance**: More aligned with how the model was trained

### 🎯 Role Structure:
- **System**: Sets the context and rules
- **User**: Represents the human asking questions  
- **Assistant**: Shows the AI's desired response format

This mimics a real conversation, making the model's learning more effective.

In [ ]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Bettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

### 🌟 Role-Based Few-Shot Example
Notice how we structure this as a realistic conversation with clear roles and consistent formatting.

## 📋 Alternative Approach: Instruction-Based Prompting

### 🔄 Instructions vs Examples

Instead of learning from examples, we can provide explicit instructions about format and content. 

### 🎯 Key Difference:
- **Examples (Few-Shot)**: The model **learns** patterns during inference
- **Instructions**: The model **follows** explicit rules and formatting guidelines

Both approaches work, but they operate differently:
- Instructions are more explicit and rule-based
- Examples create implicit learning and pattern recognition

Let's see instruction-based prompting in action:

In [ ]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answew the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

### 📝 Instruction-Based Example
Here we explicitly tell the AI exactly what information to include and how to format it.

In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Bettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

### 🧪 Comparison Test
Let's see how this compares to our few-shot approach with the same query.

# 🏷️ Practical Application: Sentiment Classification

## Real-World Use Case: Product Review Analysis

Let's apply our few-shot learning to a practical business problem: **sentiment analysis** of product reviews.

### 🎯 Business Context:
- E-commerce companies need to quickly categorize customer feedback
- Manual review analysis is time-consuming and inconsistent
- AI can provide fast, consistent sentiment classification

### 📊 Classification Categories:
- **Positive**: Happy, satisfied customers
- **Negative**: Disappointed, unsatisfied customers  
- **Neutral**: Mixed or unclear sentiment

Watch how few-shot examples help the AI understand subtle sentiment nuances!

In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Setiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

### 🎭 Sentiment Analysis in Action
Here we provide three examples showing positive, negative, and neutral sentiment patterns.

In [ ]:
context_user=[
        {"role": "system", "content": "You are an OrderBot in a fastfood restaurant."},
        {"role": "user", "content": "I have only 10 dollars, what can I order?"},
        {"role": "assistant", "content": "We have the fast menu for 7 dollars."},
        {"role": "user", "content": "Perfect! Give me one! "}
]
print(return_OAIResponse("", context_user))

### 🤖 OrderBot Conversation Example
Notice how we pass an empty string as the user message - the conversation context provides all the information needed.

# 🍔 Conversational AI: Building a Restaurant OrderBot

## Multi-Turn Conversation Design

Our final example demonstrates how to build conversational AI that maintains context across multiple exchanges.

### 🎯 Key Concepts:
- **Context Preservation**: Each message builds on previous conversation
- **Role Consistency**: The AI maintains its character as an OrderBot
- **Natural Flow**: Conversation feels human-like and helpful

### 🔍 Conversation Structure:
1. **System**: Defines the AI's role and behavior
2. **User**: Customer's initial question  
3. **Assistant**: Helpful response with suggestions
4. **User**: Customer makes a decision
5. **Assistant**: Confirms and processes the order

This simulates a real restaurant ordering experience!

# 🎓 Summary and Key Takeaways

## What We've Learned

Congratulations! You've just mastered the fundamentals of prompt engineering. Let's recap the key concepts:

### 🎯 Prompting Strategies
1. **Zero-Shot**: Quick and simple, but no format control
2. **One-Shot**: Perfect balance of simplicity and structure  
3. **Few-Shot**: Best for complex patterns and consistent formatting

### 🏆 Best Practices
- ✅ Use conversation roles (system, user, assistant) for better results
- ✅ Provide clear, specific examples that match your desired output
- ✅ Keep examples concise but representative
- ✅ Test your prompts with multiple inputs to ensure consistency

### 🚀 Real-World Applications
- **Content Generation**: Structured responses for reports, summaries
- **Classification**: Sentiment analysis, categorization, tagging
- **Conversational AI**: Chatbots, virtual assistants, customer service
- **Data Extraction**: Structured information from unstructured text

### 🔮 Next Steps
In our next notebook, we'll explore:
- **RAG (Retrieval-Augmented Generation)**: Connecting AI to external knowledge
- **LLM Agents**: Building autonomous AI systems that can take actions
- **Multi-Agent Systems**: Coordinating multiple AI agents for complex tasks

Keep practicing with different prompting strategies - the more you experiment, the better you'll become at crafting effective prompts!

---

*Happy prompting! 🎉*